<a href="https://colab.research.google.com/github/teoak/Python_Excel/blob/main/Python_in_Excel_Salary_Calculator_(Sequential_Series_Output).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import pandas
import pandas as pd

# --- Sequential Calculation Script ---
# This version removes function definitions and returns multiple values as a pandas Series.

# --- Read Inputs ---
# Directly read values and convert to float. Add basic error checks inline.
try:
    income = float(xl("B1")) # Annual Salary
except Exception as e:
    # If error, return a Series with the error message
    pd.Series({"Error": f"Error reading B1: {e}"})
    # Note: In this simplified structure, an error here will stop execution before this point.
    # However, it's good practice to handle potential returns within the try block.

try:
    pa = float(xl("E2"))
    br = float(xl("E3")) / 100.0 # Convert rate here
    ba = float(xl("E4"))
    hr = float(xl("E5")) / 100.0 # Convert rate here
    ha = float(xl("E6"))
    ar = float(xl("E7")) / 100.0 # Convert rate here
except Exception as e:
    pd.Series({"Error": f"Error reading Tax rates (E2:E7): {e}"})


try:
    pt = float(xl("H2"))
    pr = float(xl("H3")) / 100.0 # Convert rate here
    ul = float(xl("H4"))
    ur = float(xl("H5")) / 100.0 # Convert rate here
except Exception as e:
    pd.Series({"Error": f"Error reading NIC rates (H2:H5): {e}"})


# --- Perform Tax Calculation (Inline) ---
tax = 0.0 # Default tax
try:
    # Personal allowance reduction
    if income > 100000:
        reduction = (income - 100000) / 2
        effective_pa = max(0, pa - reduction)
    else:
        effective_pa = pa

    taxable_income = income - effective_pa

    # Calculate tax based on thresholds
    if taxable_income <= 0:
        tax = 0.0
    elif taxable_income <= (ba - effective_pa): # Taxed entirely at basic rate
         basic_band_limit = max(0, ba - effective_pa)
         taxable_at_basic = min(taxable_income, basic_band_limit)
         tax = taxable_at_basic * br
    elif taxable_income <= (ha - effective_pa): # Taxed at basic and higher rates
         basic_band_limit = max(0, ba - effective_pa)
         higher_band_taxable = taxable_income - basic_band_limit
         tax = (basic_band_limit * br) + (higher_band_taxable * hr)
    else: # Taxed at basic, higher, and additional rates
         basic_band_limit = max(0, ba - effective_pa)
         higher_band_limit = max(0, ha - ba) # Size of the higher rate band
         additional_band_taxable = taxable_income - basic_band_limit - higher_band_limit
         tax = (basic_band_limit * br) + (higher_band_limit * hr) + (additional_band_taxable * ar)

    tax = max(0, round(tax, 2)) # Ensure non-negative and round
except Exception as e:
     pd.Series({"Error": f"Error during Tax calculation: {e}"})


# --- Perform NIC Calculation (Inline) ---
nic = 0.0 # Default NIC
try:
    # Calculate NIC based on thresholds
    if income <= pt:
        nic = 0.0
    elif income <= ul:
        nic = (income - pt) * pr
    else: # income > ul
        nic_at_pr = (ul - pt) * pr
        nic_at_ur = (income - ul) * ur
        nic = nic_at_pr + nic_at_ur

    nic = max(0, round(nic, 2)) # Ensure non-negative and round
except Exception as e:
    pd.Series({"Error": f"Error during NIC calculation: {e}"})


# --- Calculate Net Income and Monthly Net ---
net = 0.0 # Default net
month = 0.0 # Default month
try:
    net = round((income - tax - nic), 2)
    month = round(net / 12, 2) # Calculate monthly amount
except Exception as e:
    pd.Series({"Error": f"Error calculating Net/Month: {e}"})


# --- Create and return a pandas Series with labels ---
# The index provides the labels (titles) that will appear in the first column.
# The values will appear in the second column.
results_series = pd.Series({
    "Gross Income": income,
    "Tax Deduction": tax,
    "NI Contribution": nic,
    "Net Income": net,
    "Monthly Net": month
})

# Return the Series - this is the last expression evaluated.
results_series